In [3]:
!pip install googlesearch requests
from googlesearch import search
import requests
import time

query = "Glaucoma"
for url in search(f"{query} pmc.ncbi.nlm.nih.gov/articles filetype:pdf", 20):
    try:
        if url.endswith(".pdf"):
            print(url) # Print the URL
            response = requests.get(url, timeout=10)
            if response.content and len(response.content) > 1000:
                filename = url.split("/")[-1]
                with open(filename, "wb") as f:
                    f.write(response.content)
                print("✅" + filename)
        time.sleep(1)  # Be nice to the server
    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")

print("Done!")

ERROR: Could not find a version that satisfies the requirement googlesearch (from versions: none)
ERROR: No matching distribution found for googlesearch


ModuleNotFoundError: No module named 'googlesearch'

In [ ]:
import gradio as gr
import tempfile
import os
from googlesearch import search
import requests
import time
from typing import List, Tuple

def search_and_download_pdfs(query: str, num_results: int = 20, download_dir: str = None, progress=gr.Progress()) -> List[Tuple]:
    if not download_dir:
        download_dir = tempfile.mkdtemp()
    
    results = []
    urls = list(search(f"{query} pmc.ncbi.nlm.nih.gov/articles filetype:pdf", num_results))
    
    for i, url in enumerate(progress.tqdm(urls)):
        if url.endswith(".pdf"):
            try:
                response = requests.get(url, timeout=10)
                if response.content and len(response.content) > 1000:
                    filename = url.split("/")[-1]
                    filepath = os.path.join(download_dir, filename)
                    with open(filepath, "wb") as f:
                        f.write(response.content)
                    results.append((url, filepath, "✅ Downloaded"))
                    yield results, download_dir, filepath  # Yield intermediate results
                time.sleep(1)
            except Exception as e:
                results.append((url, None, f"❌ Failed: {str(e)}"))
                yield results, download_dir, None
    
    return results, download_dir, None

def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("## PDF Search and Download")
        
        with gr.Row():
            with gr.Column(scale=3):
                query = gr.Textbox(
                    label="Search Query",
                    placeholder="Enter search terms...",
                    container=False
                )
            with gr.Column(scale=1):
                num_results = gr.Slider(
                    minimum=1,
                    maximum=50,
                    value=20,
                    step=1,
                    label="Number of Results"
                )
        
        with gr.Row():
            search_btn = gr.Button("Search and Download", variant="primary")
            clear_btn = gr.Button("Clear Results")
        
        with gr.Row():
            with gr.Column(scale=2):
                results = gr.Dataframe(
                    headers=["URL", "Local File", "Status"],
                    label="Search Results",
                    interactive=False,
                    wrap=True
                )
                download_path = gr.Textbox(
                    label="Download Directory",
                    interactive=False
                )
            
            with gr.Column(scale=1):
                pdf_viewer = gr.PDF(label="PDF Preview")
                status = gr.Status()
        
        def on_search(query_text, num):
            status.update("Starting search...")
            current_pdf = None
            for results_update, dir_path, pdf_path in search_and_download_pdfs(query_text, num):
                if pdf_path:
                    current_pdf = pdf_path
                yield {
                    results: results_update,
                    download_path: dir_path,
                    pdf_viewer: current_pdf if current_pdf else None,
                    status: f"Downloaded {len(results_update)} PDFs..."
                }
            status.update("Search completed!")
        
        def clear_outputs():
            return {
                results: None,
                download_path: None,
                pdf_viewer: None,
                status: "Cleared results"
            }

        search_btn.click(
            fn=on_search,
            inputs=[query, num_results],
            outputs=[results, download_path, pdf_viewer, status]
        )
        
        clear_btn.click(
            fn=clear_outputs,
            outputs=[results, download_path, pdf_viewer, status]
        )

    return app

if __name__ == "__main__":
    app = create_ui()
    app.launch()

In [ ]:
import gradio as gr
import tempfile
import os
from googlesearch import search
import requests
import time
from typing import List, Tuple

def search_and_download_pdfs(query: str, num_results: int = 20, download_dir: str = None, progress=gr.Progress()) -> List[Tuple]:
    if not download_dir:
        download_dir = tempfile.mkdtemp()
    
    results = []
    urls = list(search(f"{query} pmc.ncbi.nlm.nih.gov/articles filetype:pdf", num_results))
    
    for i, url in enumerate(progress.tqdm(urls)):
        if url.endswith(".pdf"):
            try:
                response = requests.get(url, timeout=10)
                if response.content and len(response.content) > 1000:
                    filename = url.split("/")[-1]
                    filepath = os.path.join(download_dir, filename)
                    with open(filepath, "wb") as f:
                        f.write(response.content)
                    results.append((url, filepath, "✅ Downloaded"))
                    yield results, download_dir, filepath  # Yield intermediate results
                time.sleep(1)
            except Exception as e:
                results.append((url, None, f"❌ Failed: {str(e)}"))
                yield results, download_dir, None
    
    return results, download_dir, None

def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("## PDF Search and Download")
        
        with gr.Row():
            with gr.Column(scale=3):
                query = gr.Textbox(
                    label="Search Query",
                    placeholder="Enter search terms...",
                    container=False
                )
            with gr.Column(scale=1):
                num_results = gr.Slider(
                    minimum=1,
                    maximum=50,
                    value=20,
                    step=1,
                    label="Number of Results"
                )
        
        with gr.Row():
            search_btn = gr.Button("Search and Download", variant="primary")
            clear_btn = gr.Button("Clear Results")
        
        with gr.Row():
            with gr.Column(scale=2):
                results = gr.Dataframe(
                    headers=["URL", "Local File", "Status"],
                    label="Search Results",
                    interactive=False,
                    wrap=True
                )
                download_path = gr.Textbox(
                    label="Download Directory",
                    interactive=False
                )
            
            with gr.Column(scale=1):
                pdf_viewer = gr.PDF(label="PDF Preview")
                status = gr.Status()
        
        def on_search(query_text, num):
            status.update("Starting search...")
            current_pdf = None
            for results_update, dir_path, pdf_path in search_and_download_pdfs(query_text, num):
                if pdf_path:
                    current_pdf = pdf_path
                yield {
                    results: results_update,
                    download_path: dir_path,
                    pdf_viewer: current_pdf if current_pdf else None,
                    status: f"Downloaded {len(results_update)} PDFs..."
                }
            status.update("Search completed!")
        
        def clear_outputs():
            return {
                results: None,
                download_path: None,
                pdf_viewer: None,
                status: "Cleared results"
            }

        search_btn.click(
            fn=on_search,
            inputs=[query, num_results],
            outputs=[results, download_path, pdf_viewer, status]
        )
        
        clear_btn.click(
            fn=clear_outputs,
            outputs=[results, download_path, pdf_viewer, status]
        )

    return app

if __name__ == "__main__":
    app = create_ui()
    app.launch()